# Создаём точку входа в Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-2.4.4-bin-hadoop2.7.tgz


In [4]:
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [6]:
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [8]:
spark

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Можно использовать только rdd.


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
loc_data = './gdrive/MyDrive/Colab Notebooks/'

In [11]:
train = sc.textFile(loc_data + 'train.csv')
test = sc.textFile(loc_data + 'test.csv')

In [12]:
import os

In [13]:
os.listdir("./gdrive/MyDrive/Colab Notebooks/")

['train.csv',
 'test.csv',
 'Untitled0.ipynb',
 'Untitled1.ipynb',
 'Table.pynb',
 'Untitled2.ipynb',
 'Updated_Test_Solopharm.ipynb',
 'Untitled3.ipynb',
 'lesson_3.ipynb',
 'Копия блокнота "Morozov_Pyspark_Test.ipynb"',
 'Morozov_Pyspark_Test.ipynb',
 'Копия блокнота "coding test ozon.ipynb""',
 'Копия блокнота "Koalas"',
 'Копия блокнота "Custom Transformer.ipynb"',
 'Копия блокнота "Custom Estimator.ipynb"',
 'Morozov_l2_HW rdd df"']

In [14]:
import pyspark.sql.functions as F

In [15]:
# !ls -la gdrive/MyDrive/'Colab Notebooks'/
!ls -la 

total 224724
drwxr-xr-x  1 root root      4096 Dec  8 17:13 .
drwxr-xr-x  1 root root      4096 Dec  8 17:10 ..
drwxr-xr-x  4 root root      4096 Dec  3 14:33 .config
drwx------  5 root root      4096 Dec  8 17:13 gdrive
drwxr-xr-x  1 root root      4096 Dec  3 14:33 sample_data
drwxr-xr-x 13 1000 1000      4096 Aug 27  2019 spark-2.4.4-bin-hadoop2.7
-rw-r--r--  1 root root 230091034 Aug 27  2019 spark-2.4.4-bin-hadoop2.7.tgz


In [16]:
# Выведите содержимое каждого файла (часть строк)
###############
# Ваш код здесь
train.take(5)

###############

['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 '842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1',
 '1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2',
 '563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2',
 '615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2']

In [17]:
# Ваш код здесь
test.take(5)

['id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi',
 '1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0',
 '2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0',
 '3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1',
 '4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0']

##  Преобразуем train и test

In [18]:
train_first_row = train.first()

train_rendered = train\
    .filter(lambda row: row != train_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')])

# len(train_rendered.collect())
train_rendered.take(5)    

[[842.0,
  0.0,
  2.2,
  0.0,
  1.0,
  0.0,
  7.0,
  0.6,
  188.0,
  2.0,
  2.0,
  20.0,
  756.0,
  2549.0,
  9.0,
  7.0,
  19.0,
  0.0,
  0.0,
  1.0,
  1.0],
 [1021.0,
  1.0,
  0.5,
  1.0,
  0.0,
  1.0,
  53.0,
  0.7,
  136.0,
  3.0,
  6.0,
  905.0,
  1988.0,
  2631.0,
  17.0,
  3.0,
  7.0,
  1.0,
  1.0,
  0.0,
  2.0],
 [563.0,
  1.0,
  0.5,
  1.0,
  2.0,
  1.0,
  41.0,
  0.9,
  145.0,
  5.0,
  6.0,
  1263.0,
  1716.0,
  2603.0,
  11.0,
  2.0,
  9.0,
  1.0,
  1.0,
  0.0,
  2.0],
 [615.0,
  1.0,
  2.5,
  0.0,
  0.0,
  0.0,
  10.0,
  0.8,
  131.0,
  6.0,
  9.0,
  1216.0,
  1786.0,
  2769.0,
  16.0,
  8.0,
  11.0,
  1.0,
  0.0,
  0.0,
  2.0],
 [1821.0,
  1.0,
  1.2,
  0.0,
  13.0,
  1.0,
  44.0,
  0.6,
  141.0,
  2.0,
  14.0,
  1208.0,
  1212.0,
  1411.0,
  8.0,
  2.0,
  15.0,
  1.0,
  1.0,
  0.0,
  1.0]]

In [19]:
# Преобразуйте test
###############
test_first_row = test.first()

test_rendered = test\
    .filter(lambda row: row != test_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')[1:]])

# len(test_rendered.collect())

test_rendered.take(5)

###############

[[1043.0,
  1.0,
  1.8,
  1.0,
  14.0,
  0.0,
  5.0,
  0.1,
  193.0,
  3.0,
  16.0,
  226.0,
  1412.0,
  3476.0,
  12.0,
  7.0,
  2.0,
  0.0,
  1.0,
  0.0],
 [841.0,
  1.0,
  0.5,
  1.0,
  4.0,
  1.0,
  61.0,
  0.8,
  191.0,
  5.0,
  12.0,
  746.0,
  857.0,
  3895.0,
  6.0,
  0.0,
  7.0,
  1.0,
  0.0,
  0.0],
 [1807.0,
  1.0,
  2.8,
  0.0,
  1.0,
  0.0,
  27.0,
  0.9,
  186.0,
  3.0,
  4.0,
  1270.0,
  1366.0,
  2396.0,
  17.0,
  10.0,
  10.0,
  0.0,
  1.0,
  1.0],
 [1546.0,
  0.0,
  0.5,
  1.0,
  18.0,
  1.0,
  25.0,
  0.5,
  96.0,
  8.0,
  20.0,
  295.0,
  1752.0,
  3893.0,
  10.0,
  0.0,
  7.0,
  1.0,
  1.0,
  0.0],
 [1434.0,
  0.0,
  1.4,
  0.0,
  11.0,
  1.0,
  49.0,
  0.5,
  108.0,
  6.0,
  18.0,
  749.0,
  810.0,
  1773.0,
  15.0,
  8.0,
  7.0,
  1.0,
  0.0,
  1.0]]

## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.html#rdd-apis)

PS: нужно сделать средсвтвами rdd pd.concat([train, test,], axis=0)

In [20]:
# Объедините train и test

###############

data = sc.union([train_rendered, test_rendered ])

###############

In [21]:
# Проверим data
for el in data.collect():
  print(el)



[842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0, 1.0]
[1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0, 2.0]
[563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0, 2.0]
[615.0, 1.0, 2.5, 0.0, 0.0, 0.0, 10.0, 0.8, 131.0, 6.0, 9.0, 1216.0, 1786.0, 2769.0, 16.0, 8.0, 11.0, 1.0, 0.0, 0.0, 2.0]
[1821.0, 1.0, 1.2, 0.0, 13.0, 1.0, 44.0, 0.6, 141.0, 2.0, 14.0, 1208.0, 1212.0, 1411.0, 8.0, 2.0, 15.0, 1.0, 1.0, 0.0, 1.0]
[1859.0, 0.0, 0.5, 1.0, 3.0, 0.0, 22.0, 0.7, 164.0, 1.0, 7.0, 1004.0, 1654.0, 1067.0, 17.0, 1.0, 10.0, 1.0, 0.0, 0.0, 1.0]
[1821.0, 0.0, 1.7, 0.0, 4.0, 1.0, 10.0, 0.8, 139.0, 8.0, 10.0, 381.0, 1018.0, 3220.0, 13.0, 8.0, 18.0, 1.0, 0.0, 1.0, 3.0]
[1954.0, 0.0, 0.5, 1.0, 0.0, 0.0, 24.0, 0.8, 187.0, 4.0, 0.0, 512.0, 1149.0, 700.0, 16.0, 3.0, 5.0, 1.0, 1.0, 1.0, 0.0]
[1445.0, 1.0, 0.5, 0.0, 0.0, 

In [22]:
train_first_row.split(',')

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [23]:
test_first_row.split(',')[1:]

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi']

## Рассчитайте соотношение сторон телефона и экрана

In [24]:
# Выведите отсортированное распределение соотношений сторон экрана
# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)


# Представим сводно сколько ширкоформатных и сколько не_широкоформатных
from operator import add
data2 = data.map(lambda row: ((0 if float(row[15]) == 0 else ('no_wide' if float(row[14])/float(row[15]) <= (16/9) else 'wide')), 1)).reduceByKey(add).collect()
# Get a result ratio
data2[1:]


[('no_wide', 1191), ('wide', 1517)]

In [25]:
# Выведите отсортированное распределение соотношений сторон экрана
# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)

###############
# Получаем отсортированные сводные данные для WIDE
###############


# from operator import add
# .filter(lambda x: x == 'wide')\
data2 = data.map(lambda row: ((0 if float(row[15]) == 0 else ('no_wide' if float(row[14])/float(row[15]) <= (16/9) else 'wide')) , round(0 if float(row[15]) == 0 else float(row[14])/float(row[15]), ndigits=1)))\
  .filter(lambda row: row[0] == 'wide')\
  .map(lambda row:(row[1], 1))\
  .reduceByKey(lambda a,b: a + b)\
  .sortBy(lambda row: row[0])\
  .collect()

data2

[(1.8, 46),
 (1.9, 60),
 (2.0, 123),
 (2.1, 39),
 (2.2, 75),
 (2.3, 58),
 (2.4, 44),
 (2.5, 66),
 (2.6, 33),
 (2.7, 36),
 (2.8, 42),
 (3.0, 100),
 (3.2, 40),
 (3.3, 14),
 (3.4, 10),
 (3.5, 35),
 (3.6, 10),
 (3.7, 14),
 (3.8, 15),
 (4.0, 50),
 (4.2, 18),
 (4.3, 17),
 (4.5, 37),
 (4.7, 14),
 (4.8, 8),
 (5.0, 68),
 (5.3, 13),
 (5.5, 19),
 (5.7, 14),
 (6.0, 70),
 (6.3, 15),
 (6.5, 12),
 (7.0, 43),
 (7.5, 14),
 (8.0, 34),
 (8.5, 15),
 (9.0, 31),
 (9.5, 9),
 (10.0, 26),
 (11.0, 19),
 (12.0, 16),
 (13.0, 14),
 (14.0, 19),
 (15.0, 11),
 (16.0, 14),
 (17.0, 18),
 (18.0, 9),
 (19.0, 10)]

In [26]:
###############
# Получаем отсортированные сводные данные для NO_WIDE
###############

data2 = data.map(lambda row: ((0 if float(row[15]) == 0 else ('no_wide' if float(row[14])/float(row[15]) <= (16/9) else 'wide')) , round(0 if float(row[15]) == 0 else float(row[14])/float(row[15]), ndigits=1)))\
  .filter(lambda row: row[0] == 'no_wide')\
  .map(lambda row:(row[1], 1))\
  .reduceByKey(lambda a,b: a + b)\
  .sortBy(lambda row: row[0])\
  .collect()

data2

[(1.1, 234),
 (1.2, 265),
 (1.3, 140),
 (1.4, 151),
 (1.5, 131),
 (1.6, 96),
 (1.7, 118),
 (1.8, 56)]

In [27]:
# Выведите отсортированное распределение плотности пикселей
# точек на дюйм (ppi)
###############
# Получаем сводно распределение по плотности, тут осортировано по сводному значению - к-ву
###############

data3 = data.map(lambda row: ((round((((float(row[11])**2 + float(row[12])**2)**0.5) / (((float(row[14])**2 + float(row[15])**2)**0.5)/2.54)), ndigits=0)), 1))\
  .reduceByKey(lambda a, b:  a + b)\
  .sortBy(lambda row: row[1])\
  .collect()

data3

# data.take(3)

[(396.0, 1),
 (612.0, 1),
 (712.0, 1),
 (68.0, 1),
 (696.0, 1),
 (980.0, 1),
 (804.0, 1),
 (924.0, 1),
 (916.0, 1),
 (608.0, 1),
 (632.0, 1),
 (1108.0, 1),
 (764.0, 1),
 (956.0, 1),
 (692.0, 1),
 (736.0, 1),
 (516.0, 1),
 (792.0, 1),
 (772.0, 1),
 (600.0, 1),
 (584.0, 1),
 (1056.0, 1),
 (508.0, 1),
 (828.0, 1),
 (616.0, 1),
 (552.0, 1),
 (748.0, 1),
 (624.0, 1),
 (476.0, 1),
 (524.0, 1),
 (56.0, 1),
 (724.0, 1),
 (668.0, 1),
 (896.0, 1),
 (644.0, 1),
 (932.0, 1),
 (496.0, 1),
 (700.0, 1),
 (800.0, 1),
 (704.0, 1),
 (1024.0, 1),
 (676.0, 1),
 (652.0, 1),
 (1040.0, 1),
 (456.0, 1),
 (64.0, 1),
 (580.0, 1),
 (880.0, 1),
 (840.0, 1),
 (564.0, 1),
 (576.0, 1),
 (520.0, 1),
 (664.0, 1),
 (744.0, 1),
 (324.0, 1),
 (528.0, 1),
 (656.0, 1),
 (640.0, 1),
 (725.0, 1),
 (477.0, 1),
 (1029.0, 1),
 (805.0, 1),
 (69.0, 1),
 (505.0, 1),
 (1105.0, 1),
 (529.0, 1),
 (789.0, 1),
 (809.0, 1),
 (545.0, 1),
 (537.0, 1),
 (445.0, 1),
 (709.0, 1),
 (1005.0, 1),
 (821.0, 1),
 (1253.0, 1),
 (489.0, 1),
 (533.0,

# JOIN
Повторите вышеописанное задание с помощью одной из функций ниже (отдельно рассчитайте для train и test, затем объедините результат)

Пример для двух RDDs: (rdd = {(1, 2), (3, 4), (3, 6)} other = {(3, 9)})

Имя функции |	Purpose |	Example |	Result
------------- |	------- |	------- |	------
subtractByKey |Remove elements with a key present in the other RDD.| rdd.subtractByKey(other) | {(1, 2)}
join | Perform an inner join between two RDDs. | rdd.join(other) | {(3, (4, 9)), (3, (6, 9))}
rightOuterJoin | Perform a join between two RDDs where the key must be present in the first RDD. | rdd.rightOuterJoin(other) | {(3,(Some(4),9)), (3,(Some(6),9))}leftOuterJoin | Perform a join between two RDDs where the key must be present in the other RDD. | rdd.leftOuterJoin(other) | {(1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9)))}
cogroup | Group data from both RDDs sharing the same key. | rdd.cogroup(other) | {(1,([2],[])), (3,([4, 6],[9]))}


In [28]:
###############
# Делаем с train , смотрим размер сведенного RDD
###############

train_for_join = train_rendered.map(lambda row: ((round((((float(row[11])**2 + float(row[12])**2)**0.5) / (((float(row[14])**2 + float(row[15])**2)**0.5)/2.54)), ndigits=0)), 1))\
  .reduceByKey(lambda a, b:  a + b)\
  .sortBy(lambda row: row[1])\
  
  
len(train_for_join.collect())


612

In [29]:
###############
# Делаем с test ,смотрим размер сведенного RDD
###############

test_for_join = test_rendered.map(lambda row: ((round((((float(row[11])**2 + float(row[12])**2)**0.5) / (((float(row[14])**2 + float(row[15])**2)**0.5)/2.54)), ndigits=0)), 1))\
  .reduceByKey(lambda a, b:  a + b)\
  .sortBy(lambda row: row[1])\

  

len(test_for_join.collect())

488

In [30]:
###############
# Объединяем результаты
###############

data3 = train_for_join\
  .join(test_for_join)\
  .sortBy(lambda x: x[0])\
  .collect()

data3





[(61.0, (1, 2)),
 (73.0, (1, 2)),
 (74.0, (1, 1)),
 (76.0, (1, 1)),
 (79.0, (4, 2)),
 (80.0, (1, 4)),
 (82.0, (4, 2)),
 (83.0, (4, 4)),
 (84.0, (5, 1)),
 (85.0, (3, 1)),
 (86.0, (2, 1)),
 (87.0, (4, 4)),
 (88.0, (2, 3)),
 (89.0, (2, 3)),
 (90.0, (3, 1)),
 (91.0, (2, 1)),
 (92.0, (4, 2)),
 (93.0, (2, 4)),
 (94.0, (2, 1)),
 (95.0, (3, 1)),
 (97.0, (2, 3)),
 (98.0, (2, 2)),
 (99.0, (5, 1)),
 (100.0, (3, 2)),
 (102.0, (5, 3)),
 (103.0, (5, 2)),
 (105.0, (5, 1)),
 (106.0, (2, 1)),
 (107.0, (4, 1)),
 (108.0, (1, 1)),
 (109.0, (4, 3)),
 (110.0, (5, 3)),
 (111.0, (3, 2)),
 (112.0, (3, 5)),
 (113.0, (2, 1)),
 (114.0, (5, 3)),
 (115.0, (2, 3)),
 (116.0, (5, 1)),
 (117.0, (5, 3)),
 (118.0, (6, 3)),
 (119.0, (7, 1)),
 (120.0, (5, 4)),
 (121.0, (6, 5)),
 (122.0, (11, 2)),
 (123.0, (4, 2)),
 (124.0, (4, 3)),
 (126.0, (4, 2)),
 (127.0, (6, 2)),
 (128.0, (9, 3)),
 (129.0, (2, 3)),
 (130.0, (1, 1)),
 (131.0, (3, 1)),
 (132.0, (8, 2)),
 (133.0, (7, 1)),
 (134.0, (3, 2)),
 (136.0, (7, 2)),
 (137.0, (8, 4

# DataFrame


In [122]:
# Считываем и объединяем данные
# Приведите все данные к правильному типу, либо считайе сразу верно
# Создаём колонки с соотношением сторон и плотностью пикселей


# Считываем

train_df = spark.read.csv(loc_data + 'train.csv', header=True, inferSchema=True)
test_df = spark.read.csv(loc_data + 'test.csv', header=True, inferSchema=True)



In [123]:
train_df.show(3)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [124]:
test_df.show(3)

+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
| id|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|
+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|  1|         1043|   1|        1.8|       1| 14|     0|         5|  0.1|      193|      3| 16|      226|    1412|3476|  12|   7|        2|      0|           1|   0|
|  2|          841|   1|        0.5|       1|  4|     1|        61|  0.8|      191|      5| 12|      746|     857|3895|   6|   0|        7|      1|           0|   0|
|  3|         1807|   1|        2.8|       0|  1|     0|        27|  0.9|      186|      3|  4|     1270|    1366|2396|  17|  10|       10|      0|           1|   1|
+---

In [125]:
# Приводим к правильному типу
train_df_correct = train_df.drop('price_range')
test_df_correct = test_df.drop('id')


In [126]:
# Проверка семпла данных
train_df_correct.show(3)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|        9|      1|           1|   0|
+-------------+----+--------

In [127]:
# Проверка семпла данных
test_df_correct.show(3)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|         1043|   1|        1.8|       1| 14|     0|         5|  0.1|      193|      3| 16|      226|    1412|3476|  12|   7|        2|      0|           1|   0|
|          841|   1|        0.5|       1|  4|     1|        61|  0.8|      191|      5| 12|      746|     857|3895|   6|   0|        7|      1|           0|   0|
|         1807|   1|        2.8|       0|  1|     0|        27|  0.9|      186|      3|  4|     1270|    1366|2396|  17|  10|       10|      0|           1|   1|
+-------------+----+--------

In [128]:
# Вывод схемы справочно
test_df_correct.printSchema()

root
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)



In [129]:
import pyspark.sql.functions as F

In [130]:
# Объединяем подготовленные DF
data4 = test_df_correct.union(train_df_correct)

In [131]:
# Проверка к-ва строк
len(data4.collect())

3000

In [132]:
data4.show(5)

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|         1043|   1|        1.8|       1| 14|     0|         5|  0.1|      193|      3| 16|      226|    1412|3476|  12|   7|        2|      0|           1|   0|
|          841|   1|        0.5|       1|  4|     1|        61|  0.8|      191|      5| 12|      746|     857|3895|   6|   0|        7|      1|           0|   0|
|         1807|   1|        2.8|       0|  1|     0|        27|  0.9|      186|      3|  4|     1270|    1366|2396|  17|  10|       10|      0|           1|   1|
|         1546|   0|        

In [137]:
# Создаём колонки с соотношением сторон и плотностью пикселей
data5 = data4.select(F.col('px_width'), F.col('px_height'), (F.col('px_width') / F.col('px_height'))\
                     .alias('wide_type'), (((F.col('px_height')**2 + F.col('px_width')**2)**0.5) / (((F.col('sc_h')**2 + F.col('sc_w')**2)**0.5)/2.54))\
                     .alias('density'))

In [146]:
# Data семпл для проверки
data5.show(5)

+--------+---------+------------------+------------------+
|px_width|px_height|         wide_type|           density|
+--------+---------+------------------+------------------+
|    1412|      226| 6.247787610619469| 261.4463627215167|
|     857|      746| 1.148793565683646|480.99404575894243|
|    1366|     1270|1.0755905511811024|240.20228926388117|
|    1752|      295| 5.938983050847457| 451.2722293294813|
|     810|      749| 1.081441922563418|164.83449612506107|
+--------+---------+------------------+------------------+
only showing top 5 rows



In [150]:
# Приводим к нормальному виду расчетные столбцы
data6 = data5.select(F.col('px_width'), F.col('px_height'), F.round('wide_type', scale=1)\
                     .alias('wide_type'), F.round('density', scale=0)\
                     .alias('density'))
data6.show()

+--------+---------+---------+-------+
|px_width|px_height|wide_type|density|
+--------+---------+---------+-------+
|    1412|      226|      6.2|  261.0|
|     857|      746|      1.1|  481.0|
|    1366|     1270|      1.1|  240.0|
|    1752|      295|      5.9|  451.0|
|     810|      749|      1.1|  165.0|
|     939|      569|      1.7|  228.0|
|    1374|     1283|      1.1|  338.0|
|    1880|     1312|      1.4|  800.0|
|     876|      556|      1.6|  185.0|
|    1009|       52|     19.4|  428.0|
|    1336|      503|      2.7|  246.0|
|    1671|      235|      7.1|  252.0|
|    1893|      829|      2.3|  830.0|
|    1856|      227|      8.2|  365.0|
|    1323|     1315|      1.0|  263.0|
|    1832|      185|      9.9|  454.0|
|    1511|      613|      2.5|  310.0|
|     532|      335|      1.6|  173.0|
|    1771|       82|     21.6|  200.0|
|     559|       47|     11.9|  285.0|
+--------+---------+---------+-------+
only showing top 20 rows



In [160]:
# Распределение по wide_type
data6.select('wide_type').groupby('wide_type').count().sort(F.col('count').desc()).show()

+---------+-----+
|wide_type|count|
+---------+-----+
|      1.1|  269|
|      1.2|  206|
|      1.3|  182|
|      1.4|  151|
|      1.5|  148|
|      1.0|  147|
|      1.6|  122|
|      1.8|  110|
|      1.7|  105|
|      2.0|   91|
|      2.2|   71|
|      2.1|   69|
|      1.9|   69|
|      2.4|   59|
|      2.3|   48|
|      2.7|   43|
|      2.5|   43|
|      2.8|   37|
|      2.6|   35|
|      3.2|   34|
+---------+-----+
only showing top 20 rows



In [161]:
# Распределение по плотности пикселей
data6.select('density').groupby('density').count().sort(F.col('count').desc()).show()

+-------+-----+
|density|count|
+-------+-----+
|  209.0|   18|
|  225.0|   17|
|  292.0|   16|
|  258.0|   15|
|  197.0|   15|
|  138.0|   15|
|  199.0|   14|
|  176.0|   13|
|  265.0|   13|
|  122.0|   13|
|  246.0|   13|
|  188.0|   13|
|  214.0|   13|
|  242.0|   13|
|  285.0|   13|
|  181.0|   12|
|  128.0|   12|
|  249.0|   12|
|  206.0|   12|
|  180.0|   12|
+-------+-----+
only showing top 20 rows



## Сохранение


In [ ]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и временем разговора в формате "1day 1hour 1minute"
# 2 балла
###############
# Ваш код здесь
###############

In [164]:
# Пересобираем dataframe, т.к. нужен df c id
data7 = test_df.union(train_df)
# train_df
# test_df
data7.show()


+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
| id|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|
+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+
|  1|         1043| 1.0|        1.8|       1| 14|     0|       5.0|  0.1|      193|      3| 16|      226|    1412|3476|  12|   7|        2|      0|           1|   0|
|  2|          841| 1.0|        0.5|       1|  4|     1|      61.0|  0.8|      191|      5| 12|      746|     857|3895|   6|   0|        7|      1|           0|   0|
|  3|         1807| 1.0|        2.8|       0|  1|     0|      27.0|  0.9|      186|      3|  4|     1270|    1366|2396|  17|  10|       10|      0|           1|   1|
|  4

In [186]:
# Выполняем расчеты с новым df с id и с временем разговора в формате "days or hours or minutes"
data8 = data7.select(F.col('id'),  F.round((((F.col('px_height')**2 + F.col('px_width')**2)**0.5) / (((F.col('sc_h')**2 + F.col('sc_w')**2)**0.5)/2.54)), scale=0)\
                     .alias('density'), F.col('talk_time'), (F.concat_ws(' or ', F.substring((F.col('talk_time')/24), 1, 1), F.col('talk_time'),F.col('talk_time')*60 ))\
                     .alias('days or hours or minutes'))
data8.show(5)

+---+-------+---------+------------------------+
| id|density|talk_time|days or hours or minutes|
+---+-------+---------+------------------------+
|  1|  261.0|        2|           0 or 2 or 120|
|  2|  481.0|        7|           0 or 7 or 420|
|  3|  240.0|       10|          0 or 10 or 600|
|  4|  451.0|        7|           0 or 7 or 420|
|  5|  165.0|        7|           0 or 7 or 420|
+---+-------+---------+------------------------+
only showing top 5 rows



In [191]:
# Сохраняем в CSV
data8.write.option("header", "true")\
        .option("delimiter",";")\
        .option("encoding", "cp1251" )\
        .csv(loc_data + 'result.csv')

In [206]:
# Проверим директорию
!ls -la './gdrive/MyDrive/Colab Notebooks/'

total 1214
-rw------- 1 root root  43567 Mar 22  2021  lesson_3.ipynb
-rw------- 1 root root 452341 Dec  8 19:44 'Morozov_l2_HW rdd df"'
-rw------- 1 root root  14244 Nov  6 14:05  Morozov_Pyspark_Test.ipynb
drwx------ 2 root root   4096 Dec  8 19:35  result.csv
-rw------- 1 root root   4968 Aug 12  2020  Table.pynb
-rw------- 1 root root  63850 Sep 26  2019  test.csv
-rw------- 1 root root 122403 Sep 26  2019  train.csv
-rw------- 1 root root    815 Feb 26  2020  Untitled0.ipynb
-rw------- 1 root root   4178 Aug  6  2020  Untitled1.ipynb
-rw------- 1 root root    421 Sep 28  2020  Untitled2.ipynb
-rw------- 1 root root    391 Mar 10  2021  Untitled3.ipynb
-rw------- 1 root root  27348 Dec  8 19:44  Updated_Test_SLF.ipynb
-rw------- 1 root root  44653 Nov  8 06:12 'Копия блокнота "coding test ozon.ipynb""'
-rw------- 1 root root  24866 Nov 18 17:59 'Копия блокнота "Custom Estimator.ipynb"'
-rw------- 1 root root 208570 Nov 18 17:41 'Копия блокнота "Custom Transformer.ipynb"'
-rw-------

In [207]:
# Проверим семпл CSV
result = spark.read\
    .options(delimiter=';')\
    .csv(loc_data + 'result.csv', header=True, inferSchema=True)
result.show(10)

+----+-------+---------+------------------------+
|  id|density|talk_time|days or hours or minutes|
+----+-------+---------+------------------------+
| 842|  334.0|        0|             0 or 0 or 0|
|1021| 1100.0|        1|            0 or 1 or 60|
| 563|  859.0|        1|            0 or 1 or 60|
| 615|  615.0|        1|            0 or 1 or 60|
|1821|  312.0|        1|            0 or 1 or 60|
|1859|  497.0|        1|            0 or 1 or 60|
|1821|  435.0|        1|            0 or 1 or 60|
|1954|  586.0|        1|            0 or 1 or 60|
|1445|  175.0|        1|            0 or 1 or 60|
| 509|  216.0|        1|            0 or 1 or 60|
+----+-------+---------+------------------------+
only showing top 10 rows

